In [1]:
# Question 6: Implement an agglomerative hierarchical clustering on a non-Euclidian space (e.g.,textual data) using Gower's distance.

In [ ]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
from collections import defaultdict

def gowers_distance_for_text(doc1_word_set, doc2_word_set, all_unique_words):
    intersection = doc1_word_set.intersection(doc2_word_set)
    union = doc1_word_set.union(doc2_word_set)
    
    if len(union) == 0:
        return 0.0
    return 1 - (len(intersection) / len(union))

def calculate_gowers_distance_matrix(documents):
    num_docs = len(documents)
    distance_matrix = np.zeros((num_docs, num_docs))
    
    document_word_sets = [set(doc) for doc in documents]
    
    all_unique_words = set(word for doc_set in document_word_sets for word in doc_set)

    for i in range(num_docs):
        for j in range(i + 1, num_docs):
            dist = gowers_distance_for_text(
                document_word_sets[i], 
                document_word_sets[j], 
                all_unique_words
            )
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist
    return distance_matrix

documents = [
    ["apple", "banana", "orange"],
    ["apple", "grape", "kiwi"],
    ["banana", "orange", "strawberry"],
    ["grape", "kiwi", "pineapple"],
    ["apple", "banana", "kiwi"],
    ["strawberry", "blueberry"],
    ["apple", "orange"],
]

gowers_dist_matrix = calculate_gowers_distance_matrix(documents)
print("Gower's Distance Matrix (lower triangle):\n", gowers_dist_matrix[np.triu_indices(len(documents), k=1)])

condensed_dist_matrix = gowers_dist_matrix[np.triu_indices(len(documents), k=1)]
linked_matrix = linkage(condensed_dist_matrix, method='average')

num_clusters = 3
clusters = fcluster(linked_matrix, num_clusters, criterion='maxclust')
print(f"\nClustering Results (into {num_clusters} clusters):")
clustered_docs = defaultdict(list)
for i, cluster_id in enumerate(clusters):
    clustered_docs[cluster_id].append(documents[i])
    print(f"Document {i+1}: {documents[i]} -> Cluster {cluster_id}")
print("\nDocuments in each cluster:")
for cluster_id, docs_in_cluster in clustered_docs.items():
    print(f"Cluster {cluster_id}:")
    for doc in docs_in_cluster:
        print(f"  {doc}")

Gower's Distance Matrix (lower triangle):
 [0.8        0.5        1.         0.5        1.         0.33333333
 1.         0.5        0.5        1.         0.75       1.
 0.8        0.75       0.75       0.8        1.         1.
 1.         0.75       1.        ]

Clustering Results (into 3 clusters):
Document 1: ['apple', 'banana', 'orange'] -> Cluster 1
Document 2: ['apple', 'grape', 'kiwi'] -> Cluster 2
Document 3: ['banana', 'orange', 'strawberry'] -> Cluster 1
Document 4: ['grape', 'kiwi', 'pineapple'] -> Cluster 2
Document 5: ['apple', 'banana', 'kiwi'] -> Cluster 2
Document 6: ['strawberry', 'blueberry'] -> Cluster 3
Document 7: ['apple', 'orange'] -> Cluster 1

Documents in each cluster:
Cluster 1:
  ['apple', 'banana', 'orange']
  ['banana', 'orange', 'strawberry']
  ['apple', 'orange']
Cluster 2:
  ['apple', 'grape', 'kiwi']
  ['grape', 'kiwi', 'pineapple']
  ['apple', 'banana', 'kiwi']
Cluster 3:
  ['strawberry', 'blueberry']
